## Import Libraries:

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4
import numpy as np
import pandas as pd
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBRegressor,XGBClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold,StratifiedKFold
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


llb


In [2]:
fulldata = pd.read_csv("../merge/fulltable_RandomForstRegressor_fill.csv")   
# fulldata = pd.read_csv("../merge/fulltable_No_fill.csv")                                                                               
print(fulldata.shape)

(40000, 462)


In [3]:
print(fulldata.isnull().sum().sort_values(ascending = False))
fulldata.head()


AMOUNT_num                   0
type_dw_more_4.0_x           0
type_dw_more_10.0            0
type_dw_more_18.0            0
type_dw_more_19.0            0
type_dw_more_23.0            0
type_dw_more_28.0_x          0
type_dw_more_31.0            0
type_dw_more_32.0            0
type_dw_less_1.0_x           0
type_dw_less_2.0_x           0
type_dw_less_4.0             0
type_dw_less_6.0_x           0
type_dw_less_7.0             0
type_dw_less_9.0             0
type_dw_less_18.0            0
type_dw_less_21.0            0
type_dw_less_24.0            0
guarantee_type_more_2.0_x    0
guarantee_type_more_4.0_x    0
guarantee_type_more_7.0      0
guarantee_type_more_10.0     0
guarantee_type_more_15.0     0
guarantee_type_more_16.0     0
guarantee_type_more_17.0     0
guarantee_type_more_19.0     0
guarantee_type_more_22.0     0
type_dw_more_6.0_x           0
finance_org_less_2.0_x       0
guarantee_type_less_2.0_x    0
                            ..
FINANCE_ORG_COUNT_zongshu    0
guarante

,REPORT_ID,AGENT_0,AGENT_1,AGENT_2,AGENT_3,AGENT_4,AGENT_5,AGENT_6,AGENT_7,AGENT_8,...,LAST_MONTHS_max,LAST_MONTHS_min,LAST_MONTHS_mean,LAST_MONTHS_zongshu,LAST_MONTHS_num,AMOUNT_max,AMOUNT_min,AMOUNT_mean,AMOUNTt_zongshu,AMOUNT_num
0,8787,0,1,0,0,0,0,0,0,0,...,0.266667,0.0,0.15,0.019643,0.035802,0.040660,0.065798,0.050971,0.008108,0.037037
1,9410,0,0,0,0,0,0,0,0,0,...,0.000000,0.0,0.00,0.008929,0.000000,0.001661,0.000295,0.001959,0.005405,0.000000
2,22999,0,1,0,0,0,0,0,0,0,...,0.000000,0.0,0.00,0.008929,0.000000,0.001661,0.000295,0.001811,0.000000,0.000000
3,24820,0,1,0,0,0,0,0,0,0,...,0.000000,0.0,0.00,0.008929,0.000000,0.001088,0.000960,0.001403,0.000000,0.000000
4,25708,1,0,0,0,0,0,0,0,0,...,0.000000,0.0,0.00,0.017857,0.012346,0.001269,0.002420,0.002157,0.008108,0.012346


In [5]:

basic_train = pd.read_csv("../input/contest_basic_train.tsv",sep = '\t')
basic_test = pd.read_csv("../input/contest_basic_test.tsv",sep = '\t')

mm = pd.concat([basic_train['REPORT_ID'],basic_train['Y']],axis = 1)
train = mm.merge(fulldata,how = 'left',left_on = 'REPORT_ID',right_on = 'REPORT_ID')
# print(train.isnull().sum().sort_values(ascending = False))

nn = pd.DataFrame(basic_test['REPORT_ID'],columns = ['REPORT_ID'])
test = nn.merge(fulldata,how = 'left',left_on = 'REPORT_ID',right_on = 'REPORT_ID')
print(train.shape,test.shape)
train.head(),test.head()

(30000, 463) (10000, 462)


(   REPORT_ID  Y  AGENT_0  AGENT_1  AGENT_2  AGENT_3  AGENT_4  AGENT_5  \
 0       8787  0        0        1        0        0        0        0   
 1       9410  0        0        0        0        0        0        0   
 2      22999  0        0        1        0        0        0        0   
 3      24820  1        0        1        0        0        0        0   
 4      25708  0        1        0        0        0        0        0   
 
    AGENT_6  AGENT_7     ...      LAST_MONTHS_max  LAST_MONTHS_min  \
 0        0        0     ...             0.266667              0.0   
 1        0        0     ...             0.000000              0.0   
 2        0        0     ...             0.000000              0.0   
 3        0        0     ...             0.000000              0.0   
 4        0        0     ...             0.000000              0.0   
 
    LAST_MONTHS_mean  LAST_MONTHS_zongshu  LAST_MONTHS_num  AMOUNT_max  \
 0              0.15             0.019643         0.035802

In [7]:
trainx = train.iloc[:,2:]
trainy = train.iloc[:,1]
print(trainx.shape,trainy.shape)
trainx.head(),trainy.head()

(30000, 461) (30000,)


(   AGENT_0  AGENT_1  AGENT_2  AGENT_3  AGENT_4  AGENT_5  AGENT_6  AGENT_7  \
 0        0        1        0        0        0        0        0        0   
 1        0        0        0        0        0        0        0        0   
 2        0        1        0        0        0        0        0        0   
 3        0        1        0        0        0        0        0        0   
 4        1        0        0        0        0        0        0        0   
 
    AGENT_8  AGENT_9     ...      LAST_MONTHS_max  LAST_MONTHS_min  \
 0        0        0     ...             0.266667              0.0   
 1        0        0     ...             0.000000              0.0   
 2        0        0     ...             0.000000              0.0   
 3        0        0     ...             0.000000              0.0   
 4        0        0     ...             0.000000              0.0   
 
    LAST_MONTHS_mean  LAST_MONTHS_zongshu  LAST_MONTHS_num  AMOUNT_max  \
 0              0.15             0

In [82]:
print(test)
trainy.value_counts()


      REPORT_ID  AGENT_0  AGENT_1  AGENT_2  AGENT_3  AGENT_4  AGENT_5  \
0       7754420        1        0        0        0        0        0   
1       8709548        0        1        0        0        0        0   
2       8753538        1        0        0        0        0        0   
3       8101125        0        0        0        0        1        0   
4       8572887        1        0        0        0        0        0   
5       5384111        0        1        0        0        0        0   
6       7640038        1        0        0        0        0        0   
7       8919014        1        0        0        0        0        0   
8       7771552        1        0        0        0        0        0   
9       5280496        0        0        0        0        0        0   
10      8017557        0        0        0        0        0        0   
11      5956992        1        0        0        0        0        0   
12      5804272        1        0        0        0

0    28125
1     1875
Name: Y, dtype: int64

## Define a function for modeling and cross-validation
1.通过XGboost 自带的XGboostcv和earning 获得所需要的树的数目
2.其他关键参数通过 网格搜索 寻找
http://blog.csdn.net/q383700092/article/details/53763328

## Step 1- Find the number of estimators for a high learning rate

In [9]:
print("trainx.shape",trainx.shape)
print("trainy.shape",trainy.shape)
from sklearn.cross_validation import train_test_split
x_train,x_valid,y_train,y_valid = train_test_split(trainx,trainy,test_size = 0.2,random_state = 2)  
print("x_train.shape",x_train.shape)
print("x_valid.shape",x_valid.shape)
print("y_train.shape",y_train.shape)
print("y_valid.shape",y_valid.shape)

trainx.shape (30000, 461)
trainy.shape (30000,)
x_train.shape (24000, 461)
x_valid.shape (6000, 461)
y_train.shape (24000,)
y_valid.shape (6000,)


In [51]:
# x_train = trainx
# y_train = trainy
print(x_train.shape)
skf = StratifiedKFold(n_splits=5)
xgb1 = XGBClassifier(
#         booster = 'dart',
        learning_rate =0.1,
        n_estimators=5000,
        max_depth=5,
        min_child_weight=5,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        StratifiedKFold = True,          #0.866124	0.008005	0.965337	0.002286
        seed=1)
xgb_param = xgb1.get_xgb_params()
#构建稀疏矩阵，运行更快
xgtrain = xgb.DMatrix(x_train, label = y_train)
# xgtest =xgb.DMatrix(dtest[predictors].values)   
mm = xgb.cv(xgb_param, xgtrain, num_boost_round=1000, nfold=10,
    metrics='auc', early_stopping_rounds=50)
mm.iloc[-3:,:]
#0.2比例数据用于训练： 0.867883	0.006550	0.957971	0.001910   
    #seed = 1：0.867687	0.003652	0.968111	0.001480
    #seed = 3：0.866441	0.007777	0.950949	0.001576
#全部数据用于训练     0.874547	0.007870	0.959760	0.000931   5
#全部数据用于训练     0.875289	0.008347	0.949510	0.001448   10                  0.8711


#0.874704	0.011258	0.978534	0.000831   0.025  eta


(24000, 461)


,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
82,0.870601,0.012018,0.959068,0.001373
83,0.870545,0.012101,0.959745,0.001311
84,0.870749,0.012187,0.960312,0.001326


In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
# from scikitplot import plotters as skplt

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC

cv = StratifiedKFold(n_splits=5, shuffle=True)

def estimate(estimator, name='estimator'):
    auc = cross_val_score(estimator, x_train, y_train, scoring='roc_auc', cv=cv).mean()
    accuracy = cross_val_score(estimator, x_train, y_train, scoring='accuracy', cv=cv).mean()
    recall = cross_val_score(estimator, x_train, y_train, scoring='recall', cv=cv).mean()

    print("{}: auc:{:f}, recall:{:f}, accuracy:{:f}".format(name, auc, recall, accuracy))
    return auc

In [63]:
estimate(XGBClassifier(
            booster = 'dart',
            learning_rate =0.08,
            n_estimators=128,
            max_depth=4,
            min_child_weight=5,
            gamma=0.3,
            subsample=0.9,
            colsample_bytree=0.9,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,
            StratifiedKFold = True,        
            seed=1)
         , 'XGBClassifier') 

print(".......................................")
estimate(XGBClassifier(
            learning_rate =0.08,
            n_estimators=103,
            max_depth=7,
            min_child_weight=8,
            gamma=0.3,
            subsample=0.9,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,
            StratifiedKFold = True,        
            seed=1)
         , 'XGBClassifier')    
# aucc = []
# for i in range(6):
#     if i== 0:
#         aucc.append(estimate(RidgeClassifier(), 'RidgeClassifier'))        #0.7969
#     if i== 1:
#         aucc.append(estimate(LogisticRegression(), 'LogisticRegression'))   #0.797659
#     if i == 2:
#         aucc.append(estimate(RandomForestClassifier(), 'RandomForestClassifier'))  #0.714345
#     if i == 3:
#         aucc.append(estimate(AdaBoostClassifier(), 'AdaBoostClassifier'))  #0.838858
#     if i == 4:
#         aucc.append(estimate(SVC(), 'SVC'))# too long to wait
#     if i == 5:
#         aucc.append(estimate(LinearSVC(), 'LinearSVC'))
# print("最终的AUC",aucc)


#RidgeClassifier: auc:0.802545, recall:0.000000, accuracy:0.936875
# LogisticRegression: auc:0.799948, recall:0.029108, accuracy:0.935792
# RandomForestClassifier: auc:0.699755, recall:0.016521, accuracy:0.936167
# AdaBoostClassifier: auc:0.834362, recall:0.059533, accuracy:0.934541
# SVC: auc:0.698281, recall:0.000000, accuracy:0.937000
# LinearSVC: auc:0.798138, recall:0.001982, accuracy:0.936750         20%预留数据

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocess

XGBClassifier: auc:0.870386, recall:0.039735, accuracy:0.936583
.......................................


c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocess

XGBClassifier: auc:0.874258, recall:0.050331, accuracy:0.936583


c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8742580219611954

In [85]:
test.iloc[:,1:].head(),x_train.head()

(   AGENT_0  AGENT_1  AGENT_2  AGENT_3  AGENT_4  AGENT_5  AGENT_6  AGENT_7  \
 0        1        0        0        0        0        0        0        0   
 1        0        1        0        0        0        0        0        0   
 2        1        0        0        0        0        0        0        0   
 3        0        0        0        0        1        0        0        0   
 4        1        0        0        0        0        0        0        0   
 
    AGENT_8  AGENT_9     ...      LAST_MONTHS_max  LAST_MONTHS_min  \
 0        0        0     ...                  0.0             0.00   
 1        0        0     ...                  0.0             0.00   
 2        0        0     ...                  0.0             0.05   
 3        0        0     ...                  0.3             0.00   
 4        0        0     ...                  0.0             0.00   
 
    LAST_MONTHS_mean  LAST_MONTHS_zongshu  LAST_MONTHS_num  AMOUNT_max  \
 0          0.000000             0

In [61]:
print(x_train.shape)   #0.690831	0.010714	0.776624	0.002014
print(y_train.shape)
print("mm.shape[0]",mm.shape[0])
xgb1 = XGBClassifier(
            booster = 'dart',
            learning_rate =0.08,
            n_estimators=128,
            max_depth=4,
            min_child_weight=5,
            gamma=0.3,
            subsample=0.9,
            colsample_bytree=0.9,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,
            StratifiedKFold = True,        
            seed=1)
xgb1.fit(x_train,y_train)
pred1 = xgb1.predict_proba(x_valid)[:,1]
auc = roc_auc_score(y_valid,pred1)
print("xgb1 auc",auc)


xgb2 = XGBClassifier(
            learning_rate =0.08,
            n_estimators=103,
            max_depth=7,
            min_child_weight=8,
            gamma=0.3,
            subsample=0.9,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,
            StratifiedKFold = True,        
            seed=1)
xgb2.fit(x_train,y_train)
pred2 = xgb2.predict_proba(x_valid)[:,1]
# pred1 = xgb2.predict_proba(x_valid)
auc = roc_auc_score(y_valid,pred2)
print("xgb2 auc",auc)
predttal = 0.8*pred1 + 0.2*pred2#+0.01*pred5
auc = roc_auc_score(y_valid,predttal)
print("total auc",auc)

#划分数据集 种子1
# xgb1 auc 0.8811458725823234
# xgb2 auc 0.879132903372102
# total auc 0.8823769163892053

#划分数据集 种子2




(24000, 461)
(24000,)
mm.shape[0] 85
xgb1 auc 0.874961529579074
xgb2 auc 0.8756718649341809
total auc 0.8767312904911815


In [86]:
# 看下很多个随机数结果怎么样？  30个随机数
import random
print (random.uniform(0.85,0.95))  
rand = []
for i in range(30):
    rand.append(random.uniform(0.85,0.95))
print("rand",rand)
last = 0
for i in range(30):
    pred1 = 0
    xgb1 = XGBClassifier(
            booster = 'dart',
            learning_rate =0.08,
            n_estimators=128,
            max_depth=4,
            min_child_weight=5,
            gamma=0.3,
            subsample=rand[i],
            colsample_bytree=0.9,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,
            StratifiedKFold = True,        
            seed=1)
    xgb1.fit(x_train,y_train)
    pred1 = xgb1.predict_proba(test.iloc[:,1:])[:,1] 
#     pred1 = xgb1.predict_proba(x_valid)[:,1]  REPORT_ID
#     auc = roc_auc_score(y_valid,pred1)
    last += pred1
#     print("i:{},xgb1 auc:{}".format(i,auc))
xgb1 = XGBClassifier(
            booster = 'dart',
            learning_rate =0.08,
            n_estimators=128,
            max_depth=4,
            min_child_weight=5,
            gamma=0.3,
            subsample=0.9,
            colsample_bytree=0.9,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,
            StratifiedKFold = True,        
            seed=1)
xgb1.fit(x_train,y_train)
pred0 = xgb1.predict_proba(test.iloc[:,1:])[:,1]
# pred0 = xgb1.predict_proba(x_valid)[:,1]
# auc = roc_auc_score(y_valid,pred0)
# print("sumple = 0.8",auc)
last += pred0
last = last/31
# roc_auc_score(y_valid,last)

#划分数据集的种子1
# i:0,xgb1 auc:0.8821081295318076
# i:1,xgb1 auc:0.8845237903247483
# i:2,xgb1 auc:0.8869663298034288
# i:3,xgb1 auc:0.8810119678570015
# i:4,xgb1 auc:0.8838410717069578
# i:5,xgb1 auc:0.8831979380627113
# i:6,xgb1 auc:0.8825132646314127
# i:7,xgb1 auc:0.8841641046392121
# i:8,xgb1 auc:0.8814801456922506
# sumple = 0.8 0.8811458725823234
# Out[50]:
# 0.8864654088419147

# i:0,xgb1 auc:0.876384145081499
# i:1,xgb1 auc:0.8769277145045034
# i:2,xgb1 auc:0.8778522201018585
# i:3,xgb1 auc:0.8763282323054297
# i:4,xgb1 auc:0.8760491546231357
# i:5,xgb1 auc:0.8763890070620268
# i:6,xgb1 auc:0.8761916106525993
# i:7,xgb1 auc:0.8753524328135067
# i:8,xgb1 auc:0.8752795031055901
# sumple = 0.8 0.8776137399569713
# Out[29]:
# 0.8774357914696551

0.9305579439328346
rand [0.9342349077767629, 0.8767530058831151, 0.8910102304145874, 0.9194468618681169, 0.8661126591790965, 0.8750625304808892, 0.8677861896427991, 0.8963524586825469, 0.9290635643243113, 0.927015647888572, 0.8725936346721158, 0.9250058891250146, 0.900547221070918, 0.9393418956829184, 0.8730785637613113, 0.8550004961374454, 0.8510973262940745, 0.8955297167917619, 0.903463933469629, 0.8888218288560606, 0.8939986639589079, 0.9210869759259905, 0.8585517686040097, 0.9250235729920607, 0.8872113322756269, 0.8608208136836271, 0.9124973167242, 0.9076588572384732, 0.9104627584028719, 0.8987411655468898]


In [87]:
rand2 = []
for i in range(30):
    rand2.append(random.uniform(0.85,0.95))
print("rand",rand2)
last2 = 0
for i in range(30):
    pred2 = 0
    xgb2 = XGBClassifier(
            learning_rate =0.08,
            n_estimators=103,
            max_depth=7,
            min_child_weight=8,
            gamma=0.3,
            subsample=rand2[i],
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,
            StratifiedKFold = True,        
            seed=1)
    xgb2.fit(x_train,y_train)
    pred2 = xgb2.predict_proba(test.iloc[:,1:])[:,1]
#     pred2 = xgb2.predict_proba(x_valid)[:,1]
#     auc = roc_auc_score(y_valid,pred2)
    last2 += pred2
#     print("i:{},xgb1 auc:{}".format(i,auc))
xgb2 = XGBClassifier(
            learning_rate =0.08,
            n_estimators=103,
            max_depth=7,
            min_child_weight=8,
            gamma=0.3,
            subsample=0.9,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,
            StratifiedKFold = True,        
            seed=1)
xgb2.fit(x_train,y_train)
pred2 = xgb1.predict_proba(test.iloc[:,1:])[:,1]
# pred2 = xgb1.predict_proba(x_valid)[:,1]
# auc = roc_auc_score(y_valid,pred2)
# print("sumple = 0.8",auc)
last2 += pred2
last2 = last2/31
# roc_auc_score(y_valid,last2)

rand [0.9476409682544764, 0.8953775054455586, 0.8512518027087432, 0.9228951089401374, 0.9409054452116784, 0.8935882541265096, 0.9291277127033143, 0.9049650501126413, 0.9080491291854176, 0.9332104886011403, 0.8771546422699255, 0.927473064291222, 0.9044888730050088, 0.8854482865715768, 0.9444579412074745, 0.8883093960726984, 0.9476835705086385, 0.9292769758531199, 0.9005114712137648, 0.8739143626437157, 0.9069986396864625, 0.9311684317086566, 0.9187632284161857, 0.8974392165012887, 0.9426838526874581, 0.9331158494023933, 0.8654054632407047, 0.9158619949566912, 0.9364425309274242, 0.9183293962893184]


In [88]:
# dd = (0.8*predttal + 0.2*last)
dd = 0.4*last + 0.6*last2
# print(roc_auc_score(y_valid,dd))
# rr = RandomForestClassifier(oob_score=True,
#                        n_estimators = 107,
#                        max_depth=13,
#                        min_samples_split = 8,
#                        min_samples_leaf=12,
#                        max_features='sqrt' , 
#                        random_state=3)

# rr.fit(x_train,y_train)
# pred3 = rr.predict_proba(x_valid)[:,1]
# auc = roc_auc_score(y_valid,pred3)
# print("rr",auc)
# dd = 0.4*last + 0.4*last2+0.2 * pred3
# roc_auc_score(y_valid,dd)

In [90]:
dd = pd.Series(dd)
final = pd.concat([test.REPORT_ID,dd],axis = 1)
final.columns = ('REPORT_ID',"Y_predict_proba")
print(final.head())
print(final.shape)
# final.to_csv("../output/test_Y_proba.csv",index = False)

   REPORT_ID  Y_predict_proba
0    7754420         0.257419
1    8709548         0.001702
2    8753538         0.001531
3    8101125         0.002848
4    8572887         0.001493
(10000, 2)


# 下面通过网格搜索寻找其他参数

In [52]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,8,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(
#         booster = 'dart',
        learning_rate =0.1,
        n_estimators=mm.shape[0],
        max_depth=5,
        min_child_weight=5,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        StratifiedKFold = True,          #0.866124	0.008005	0.965337	0.002286
        seed=1), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(x_train,y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.86635, std: 0.00754, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.86628, std: 0.00520, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.86725, std: 0.00555, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.86798, std: 0.00717, params: {'max_depth': 3, 'min_child_weight': 7},
  mean: 0.87070, std: 0.00694, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.87104, std: 0.00684, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.87057, std: 0.00725, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.87149, std: 0.00709, params: {'max_depth': 5, 'min_child_weight': 7},
  mean: 0.86798, std: 0.00596, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.87029, std: 0.00837, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.86966, std: 0.00702, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.87170, std: 0.00568, params: {'max_depth': 7, 'min_child_weight': 7},
  mean: 0.86553, std: 0.00960, params: {

In [53]:
# #Grid seach on subsample and max_features
# #Choose all predictors except target & IDcols

print(" x_train", x_train.shape)
print("y_train",y_train.shape)
param_test2 = {
    'max_depth':[7,8,6],
    'min_child_weight':[6,7,8]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier(
#         booster = 'dart',
        learning_rate =0.1,
        n_estimators=mm.shape[0],
        max_depth=5,
        min_child_weight=5,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        StratifiedKFold = True,          #0.866124	0.008005	0.965337	0.002286
        seed=1), 
                       param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(x_train,y_train)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

 x_train (24000, 461)
y_train (24000,)


c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.86887, std: 0.00701, params: {'max_depth': 7, 'min_child_weight': 6},
  mean: 0.87170, std: 0.00568, params: {'max_depth': 7, 'min_child_weight': 7},
  mean: 0.87321, std: 0.00542, params: {'max_depth': 7, 'min_child_weight': 8},
  mean: 0.86811, std: 0.00573, params: {'max_depth': 8, 'min_child_weight': 6},
  mean: 0.86963, std: 0.00751, params: {'max_depth': 8, 'min_child_weight': 7},
  mean: 0.86878, std: 0.00876, params: {'max_depth': 8, 'min_child_weight': 8},
  mean: 0.87038, std: 0.00668, params: {'max_depth': 6, 'min_child_weight': 6},
  mean: 0.87146, std: 0.00833, params: {'max_depth': 6, 'min_child_weight': 7},
  mean: 0.87042, std: 0.00788, params: {'max_depth': 6, 'min_child_weight': 8}],
 {'max_depth': 7, 'min_child_weight': 8},
 0.8732053097918427)

In [54]:
# #Grid seach on subsample and max_features
# #Choose all predictors except target & IDcols
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
print("mm",mm.shape[0])
gsearch3 = GridSearchCV(estimator = XGBClassifier(
#         booster = 'dart',
        learning_rate =0.1,
        n_estimators=mm.shape[0],
        max_depth=7,
        min_child_weight=8,
        gamma=0.3,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        StratifiedKFold = True,        
        seed=1), 
                        param_grid = param_test3, scoring='roc_auc',n_jobs=4, cv=3)
gsearch3.fit(x_train,y_train)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

mm 85


c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.86611, std: 0.00479, params: {'gamma': 0.0},
  mean: 0.86636, std: 0.00395, params: {'gamma': 0.1},
  mean: 0.86676, std: 0.00550, params: {'gamma': 0.2},
  mean: 0.86698, std: 0.00659, params: {'gamma': 0.3},
  mean: 0.86586, std: 0.00493, params: {'gamma': 0.4}],
 {'gamma': 0.3},
 0.8669757899975065)

In [56]:
param_test4 = {
    'learning_rate':[0.02,0.025,0.05,0.08,0.1]
}
gsearch4 = GridSearchCV(estimator =  XGBClassifier(
            booster = 'dart',
            learning_rate =0.08,
            n_estimators=mm.shape[0],
            max_depth=7,
            min_child_weight=8,
            gamma=0.3,
            subsample=0.8,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,
            StratifiedKFold = True,        
            seed=1), 
                       param_grid = param_test4, scoring='roc_auc',n_jobs=3,iid=False, cv=5)
gsearch4.fit(x_train,y_train)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.86491, std: 0.00882, params: {'learning_rate': 0.02},
  mean: 0.86545, std: 0.00796, params: {'learning_rate': 0.025},
  mean: 0.87154, std: 0.00788, params: {'learning_rate': 0.05},
  mean: 0.87254, std: 0.00504, params: {'learning_rate': 0.08},
  mean: 0.87033, std: 0.00701, params: {'learning_rate': 0.1}],
 {'learning_rate': 0.08},
 0.8725358437451229)

Tune subsample and colsample_bytree

In [57]:
# #Grid seach on subsample and max_features
# #Choose all predictors except target & IDcols
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier(
#             booster = 'dart',
            learning_rate =0.08,
            n_estimators=mm.shape[0],
            max_depth=7,
            min_child_weight=8,
            gamma=0.3,
            subsample=0.8,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,
            StratifiedKFold = True,        
            seed=1), 
                       param_grid = param_test4, scoring='roc_auc',n_jobs=3,iid=False, cv=5)
gsearch4.fit(x_train,y_train)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.86727, std: 0.00830, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: 0.87183, std: 0.00575, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: 0.87172, std: 0.00702, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: 0.87314, std: 0.00821, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: 0.86790, std: 0.01025, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: 0.87231, std: 0.00904, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: 0.87281, std: 0.00772, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: 0.87402, std: 0.00816, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: 0.86991, std: 0.00732, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: 0.87006, std: 0.00750, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: 0.87245, std: 0.00748, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: 0.87455, std: 0.00637, params: {'colsample_bytree': 0.8, 'subsample'

tune subsample:

In [58]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test5 = {
    'subsample':[i/100.0 for i in range(85,95,5)],
    'colsample_bytree':[i/100.0 for i in range(75,85,5)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier(
#             booster = 'dart',
            learning_rate =0.08,
            n_estimators=mm.shape[0],
            max_depth=7,
            min_child_weight=8,
            gamma=0.3,
            subsample=0.9,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,
            StratifiedKFold = True,        
            seed=1), 
                       param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch5.fit(x_train,y_train)
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.87295, std: 0.00650, params: {'colsample_bytree': 0.75, 'subsample': 0.85},
  mean: 0.87396, std: 0.00814, params: {'colsample_bytree': 0.75, 'subsample': 0.9},
  mean: 0.87154, std: 0.00814, params: {'colsample_bytree': 0.8, 'subsample': 0.85},
  mean: 0.87455, std: 0.00637, params: {'colsample_bytree': 0.8, 'subsample': 0.9}],
 {'colsample_bytree': 0.8, 'subsample': 0.9},
 0.8745455669775234)

Got the same value as assument and no change requried.

Try regularization:  (防止过拟合的)

In [59]:
# #Grid seach on subsample and max_features
# #Choose all predictors except target & IDcols
param_test6 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier(
            booster = 'dart',
            learning_rate =0.08,
            n_estimators=mm.shape[0],
            max_depth=7,
            min_child_weight=8,
            gamma=0.3,
            subsample=0.9,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,
            StratifiedKFold = True,        
            seed=1), 
                       param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(x_train,y_train)
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.87252, std: 0.00542, params: {'reg_alpha': 1e-05},
  mean: 0.87144, std: 0.00422, params: {'reg_alpha': 0.01},
  mean: 0.87240, std: 0.00526, params: {'reg_alpha': 0.1},
  mean: 0.87146, std: 0.00462, params: {'reg_alpha': 1},
  mean: 0.83631, std: 0.00418, params: {'reg_alpha': 100}],
 {'reg_alpha': 1e-05},
 0.872520973265528)

In [ ]:
# #Grid seach on subsample and max_features
# #Choose all predictors except target & IDcols
# param_test7 = {
#     'reg_alpha':[0, 0.00001, 0.00005, 0.01, 0.05]
# }
# gsearch7 = GridSearchCV(estimator = XGBClassifier(
#             booster = 'dart',
#             learning_rate =0.08,
#             n_estimators=mm.shape[0],
#             max_depth=4,
#             min_child_weight=5,
#             gamma=0.3,
#             subsample=0.9,
#             colsample_bytree=0.9,
#             objective= 'binary:logistic',
#             nthread=4,
#             scale_pos_weight=1,
#             StratifiedKFold = True,        
#             seed=1), 
#                        param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# gsearch7.fit(x_train,y_train)
# gsearch7.grid_scores_, gsearch7.best_params_, gsearch7.best_score_

In [60]:
xgb3 = XGBClassifier(
#             booster = 'dart',
            learning_rate =0.08,
            n_estimators=103,
            max_depth=7,
            min_child_weight=8,
            gamma=0.3,
            subsample=0.9,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,
            StratifiedKFold = True,        
            seed=1)
param_test7 = {
    'n_estimators':range(mm.shape[0]-20,mm.shape[0]+20,1)
}
gsearch7 = GridSearchCV(estimator = xgb3,
                       param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch7.fit(x_train,y_train)
gsearch7.grid_scores_, gsearch7.best_params_, gsearch7.best_score_

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.87303, std: 0.00595, params: {'n_estimators': 65},
  mean: 0.87319, std: 0.00573, params: {'n_estimators': 66},
  mean: 0.87365, std: 0.00587, params: {'n_estimators': 67},
  mean: 0.87358, std: 0.00608, params: {'n_estimators': 68},
  mean: 0.87385, std: 0.00583, params: {'n_estimators': 69},
  mean: 0.87387, std: 0.00587, params: {'n_estimators': 70},
  mean: 0.87392, std: 0.00596, params: {'n_estimators': 71},
  mean: 0.87406, std: 0.00589, params: {'n_estimators': 72},
  mean: 0.87416, std: 0.00581, params: {'n_estimators': 73},
  mean: 0.87392, std: 0.00592, params: {'n_estimators': 74},
  mean: 0.87413, std: 0.00603, params: {'n_estimators': 75},
  mean: 0.87435, std: 0.00596, params: {'n_estimators': 76},
  mean: 0.87450, std: 0.00599, params: {'n_estimators': 77},
  mean: 0.87426, std: 0.00617, params: {'n_estimators': 78},
  mean: 0.87445, std: 0.00620, params: {'n_estimators': 79},
  mean: 0.87461, std: 0.00609, params: {'n_estimators': 80},
  mean: 0.87459, std: 0.

In [70]:
print(x_train.shape)
print(y_train.shape)
rr = RandomForestClassifier(oob_score=True,
                       min_samples_leaf=20,
                       max_depth=8,
                       max_features='sqrt' , 
                       random_state=3)
param_test1 = {"n_estimators":range(80,100,2)}

clfrr = GridSearchCV(estimator = rr,n_jobs = 3,param_grid = param_test1,scoring='roc_auc', cv=5)
clfrr.fit(x_train,y_train)
print( clfrr.best_params_, clfrr.best_score_) 

(24000, 461)
(24000,)
随机森林包中的self.oob_score_ 0.9370833333333334
{'n_estimators': 98} 0.8089200792699625


In [72]:
rr = RandomForestClassifier(oob_score=True,
                       n_estimators = 98,
                       max_features='sqrt' , 
                       random_state=3)


param_test1 = {'max_depth':range(11,14,1), 'min_samples_split':range(8,16,1)}
# param_test1 = {"max_depth":range(5,8,1),'min_samples_split':range(8,16,1)}

clfrr = GridSearchCV(estimator = rr,param_grid = param_test1,n_jobs = 3,scoring='roc_auc', cv=5)
clfrr.fit(x_train,y_train)
print( clfrr.best_params_, clfrr.best_score_)

随机森林包中的self.oob_score_ 0.9372916666666666
{'max_depth': 13, 'min_samples_split': 10} 0.8093558873848273


In [73]:
rr = RandomForestClassifier(oob_score=True,
                       n_estimators = 98,
                       max_depth = 13,
                       max_features='sqrt' , 
                       random_state=3)

param_test1 = { 'min_samples_leaf':range(6,13,1),'min_samples_split':range(8,15,1)}


clfrr = GridSearchCV(estimator = rr,param_grid = param_test1,n_jobs = 3,scoring='roc_auc', cv=5)
clfrr.fit(x_train,y_train)
print( clfrr.best_params_, clfrr.best_score_)

随机森林包中的self.oob_score_ 0.9370833333333334
{'min_samples_leaf': 12, 'min_samples_split': 8} 0.8154374718417898


In [74]:
print(x_train.shape)
print(y_train.shape)
rr = RandomForestClassifier(oob_score=True,
                       n_estimators = 107,
                       max_depth=13,
                       min_samples_split = 8,
                       min_samples_leaf=12,
                       max_features='sqrt' , 
                       random_state=3)
param_test1 = {"n_estimators":range(90,110,1)}

clfrr = GridSearchCV(estimator = rr,n_jobs = 3,param_grid = param_test1,scoring='roc_auc', cv=5)
clfrr.fit(x_train,y_train)
print( clfrr.best_params_, clfrr.best_score_)  #180

(24000, 461)
(24000,)
随机森林包中的self.oob_score_ 0.9370833333333334
{'n_estimators': 107} 0.8160107950847911
